In [3]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [6]:
dataPath = "segmentations/inducedRainbow/final/orange"

In [7]:
folders = [x[0] for x in os.walk(dataPath)]
folders = sorted(folders)

In [ ]:
allClusterSizes = []
for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f if "clusterSize" in f]
    for file in files:
        cluster = np.load(join(folder, file), allow_pickle=True).item()
        clusterSizes = []
        for key in cluster:
            clusterSizes.append(len(cluster[key]))
        allClusterSizes.append(np.max(clusterSizes))

maxClusterSize = np.max(allClusterSizes)
columnList = ["age", "aorta"]
for i in range(maxClusterSize):
    columnList.append(str(i+1))
dfResults = pd.DataFrame(columns=columnList)

for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f if "clusterSize" in f]
    for file in files:
        cluster = np.load(join(folder, file), allow_pickle=True).item()
        clusterSizes = []
        for key in cluster:
            clusterSizes.append(len(cluster[key]))
        clusterSizes = np.asarray(clusterSizes)
        unique, count = np.unique(clusterSizes, return_counts=True)
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        row = [age, aorta]
        for i in range(maxClusterSize):
            if count[np.where(unique == (i+1))].size == 0:
                row.append(0)
            else:
                row.append(count[np.where(unique == (i+1))][0])
        dfResults = dfResults.append(pd.DataFrame([row], columns=columnList))

In [11]:
dfResults.to_csv("results/inducedRainbow/allClusterSizesOrange.csv", index=False)